In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(
            self.template.format(a, eval(a)._repr_html_()) for a in self.args
        )

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)



In [2]:
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6]])
a

b = a.flatten()
b
c = a.flatten('F')
c

array([[1, 2, 3],
       [4, 5, 6]])

array([1, 2, 3, 4, 5, 6])

array([1, 4, 2, 5, 3, 6])

In [ ]:
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import numpy as np
from ordered_set import OrderedSet
from eos.data_io.config import drivers, trucks_by_name, drivers_by_id

# DriverSet = OrderedSet(
#     [
#         'wang-cheng',
#         'li-changlong',
#         'chen-hongmei',
#         'zheng-longfei',
#     ]
# )
# DriverSet[0]
# DriverSet.get_loc('wang-cheng')
# DriverSet


In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

ts = pd.to_datetime(datetime.now())
ts_ind = ts + pd.to_timedelta(np.arange(2), 'H')
ts_ind
df = pd.DataFrame(a, index=ts_ind, columns=['c1', 'c2', 'c3'])
df
df.index[0]


DatetimeIndex(['2023-06-09 15:17:41.649423', '2023-06-09 16:17:41.649423'], dtype='datetime64[ns]', freq=None)

,c1,c2,c3
2023-06-09 15:17:41.649423,1,2,3
2023-06-09 16:17:41.649423,4,5,6


Timestamp('2023-06-09 15:17:41.649423')

In [6]:
df_ts = pd.DataFrame(np.array([ts]), columns=['timestamp'])
df_ts

,timestamp
0,2023-06-09 15:17:41.649423


In [7]:
s = np.arange(10)
s

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Generate state

In [114]:
ss = np.arange(12)
a1 = ss[:4].tolist()
a2 = ss[4:8].tolist()
a3 = ss[8:].tolist()
ss = [a1, a2, a3]
ss
df_ss = pd.DataFrame(ss, columns=['t', 'v', 'p', 'b']).set_index('t')
df_ss

[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]

,v,p,b
t,,,
0,1,2,3
4,5,6,7
8,9,10,11


In [116]:
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame({'t': a1, 'v': a2, 'p': a3}).set_index('t')
df_ss


,v,p
t,,
0,4,8
1,5,9
2,6,10
3,7,11


In [8]:
from functools import reduce

velocity = pd.Series(s[:4], name='velocity')
thrust = pd.Series(s[4:7], name='thrust')
brake = pd.Series(s[7:], name='brake')
series = [velocity, thrust, brake]
state = reduce(
    lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
)
state = state.stack().swaplevel(0, 1).sort_index()
state.name = 'state'
state
len(state)

brake     0    7.0
          1    8.0
          2    9.0
thrust    0    4.0
          1    5.0
          2    6.0
velocity  0    0.0
          1    1.0
          2    2.0
          3    3.0
Name: state, dtype: float64

10

In [9]:
a = len(state) + np.arange(6)
ar0 = pd.Series(a[:2], name='row0')
ar1 = pd.Series(a[2:4], name='row1')
ar2 = pd.Series(a[4:], name='row2')
series = [ar0, ar1, ar2]
action = reduce(
    lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
)
action = action.stack().swaplevel(0, 1).sort_index()
action.name = 'action'
action
len(action) + len(state)

row0  0    10
      1    11
row1  0    12
      1    13
row2  0    14
      1    15
Name: action, dtype: int64

16

In [10]:
reward = pd.Series([len(state) + len(action)], name='reward')
reward

0    16
Name: reward, dtype: int64

In [11]:
velocity = pd.Series(len(state) + len(action) + len(reward) + s[:4], name='velocity')
thrust = pd.Series(len(state) + len(action) + len(reward) + s[4:7], name='thrust')
brake = pd.Series(len(state) + len(action) + len(reward) + s[7:], name='brake')
series = [velocity, thrust, brake]
nstate = reduce(
    lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
)
nstate = nstate.stack().swaplevel(0, 1).sort_index()
nstate.name = 'nstate'
nstate
len(nstate)


brake     0    24.0
          1    25.0
          2    26.0
thrust    0    21.0
          1    22.0
          2    23.0
velocity  0    17.0
          1    18.0
          2    19.0
          3    20.0
Name: nstate, dtype: float64

10

In [12]:
timestamp = pd.Series([ts], name='timestamp')
timestamp.index
state.index
action.index
reward.index
nstate.index

RangeIndex(start=0, stop=1, step=1)

MultiIndex([(   'brake', 0),
            (   'brake', 1),
            (   'brake', 2),
            (  'thrust', 0),
            (  'thrust', 1),
            (  'thrust', 2),
            ('velocity', 0),
            ('velocity', 1),
            ('velocity', 2),
            ('velocity', 3)],
           )

MultiIndex([('row0', 0),
            ('row0', 1),
            ('row1', 0),
            ('row1', 1),
            ('row2', 0),
            ('row2', 1)],
           )

RangeIndex(start=0, stop=1, step=1)

MultiIndex([(   'brake', 0),
            (   'brake', 1),
            (   'brake', 2),
            (  'thrust', 0),
            (  'thrust', 1),
            (  'thrust', 2),
            ('velocity', 0),
            ('velocity', 1),
            ('velocity', 2),
            ('velocity', 3)],
           )

In [13]:
timestamp_index = (timestamp.name, '', 0)
state_index = [(state.name, *i) for i in state.index]
reward_index = (reward.name, '', 0)
action_index = [(action.name, *i) for i in action.index]
nstate_index = [(nstate.name, *i) for i in nstate.index]

multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, reward_index, *action_index, *nstate_index]
)
multiindex


MultiIndex([('timestamp',         '', 0),
            (    'state',    'brake', 0),
            (    'state',    'brake', 1),
            (    'state',    'brake', 2),
            (    'state',   'thrust', 0),
            (    'state',   'thrust', 1),
            (    'state',   'thrust', 2),
            (    'state', 'velocity', 0),
            (    'state', 'velocity', 1),
            (    'state', 'velocity', 2),
            (    'state', 'velocity', 3),
            (   'reward',         '', 0),
            (   'action',     'row0', 0),
            (   'action',     'row0', 1),
            (   'action',     'row1', 0),
            (   'action',     'row1', 1),
            (   'action',     'row2', 0),
            (   'action',     'row2', 1),
            (   'nstate',    'brake', 0),
            (   'nstate',    'brake', 1),
            (   'nstate',    'brake', 2),
            (   'nstate',   'thrust', 0),
            (   'nstate',   'thrust', 1),
            (   'nstate',   'thrus

In [14]:
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex
observation
observation.dtype

timestamp            0    2023-06-09 15:17:41.649423
state      brake     0                           7.0
                     1                           8.0
                     2                           9.0
           thrust    0                           4.0
                     1                           5.0
                     2                           6.0
           velocity  0                           0.0
                     1                           1.0
                     2                           2.0
                     3                           3.0
reward               0                            10
action     row0      0                            11
                     1                            12
           row1      0                            13
                     1                            14
           row2      0                            15
                     1                            16
nstate     brake     0                        

dtype('O')

In [15]:

observation0 = observation.copy()
observation0.loc['timestamp', '', 0] = ts + pd.Timedelta(1, 'h')
observation1 = observation.copy()
observation1.loc['timestamp', '', 0] = ts + pd.Timedelta(2, 'h')
observation2 = observation.copy()
observation2.loc['timestamp', '', 0] = ts + pd.Timedelta(3, 'h')
observation3 = observation.copy()
observation3.loc['timestamp', '', 0] = ts + pd.Timedelta(4, 'h')
observation4 = observation.copy()
observation4.loc['timestamp', '', 0] = ts + pd.Timedelta(5, 'h')
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]


In [16]:

dfs_epi = pd.concat(observation_list, axis=1).transpose()
display('dfs_epi')
dfs_epi.columns

dfs_epi
                   timestamp state                                            
                             brake           thrust           velocity        
                           0     0    1    2      0    1    2        0    1   
0 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0  \
1 2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   
2 2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   
3 2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   
4 2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   

        ... nstate                                                            
        ...  brake             thrust             velocity                    
     2  ...      0     1     2      0     1     2        0     1     2     3  
0  2.0  ...   24.0  25.0  26.0   21.0  22.0  23.0     17.0  18.0  19.0  20.0  
1  2.0  ...   24.0  25.0  26.0   21.0  22.0  23.0     17.0  18.0  19.0  20.0  
2  2.0  ...   24.0  25.0  26.0   21.0  22.0  23.0     17.0  18.0  19.0  20.0  
3  2.0  ...   24.0  25.0  26.0   21.0  22.0  23.0     17.0  18.0  19.0  20.0  
4  2.0  ...   24.0  25.0  26.0   21.0  22.0  23.0     17.0  18.0  19.0  20.0  

[5 rows x 28 columns]

MultiIndex([('timestamp',         '', 0),
            (    'state',    'brake', 0),
            (    'state',    'brake', 1),
            (    'state',    'brake', 2),
            (    'state',   'thrust', 0),
            (    'state',   'thrust', 1),
            (    'state',   'thrust', 2),
            (    'state', 'velocity', 0),
            (    'state', 'velocity', 1),
            (    'state', 'velocity', 2),
            (    'state', 'velocity', 3),
            (   'reward',         '', 0),
            (   'action',     'row0', 0),
            (   'action',     'row0', 1),
            (   'action',     'row1', 0),
            (   'action',     'row1', 1),
            (   'action',     'row2', 0),
            (   'action',     'row2', 1),
            (   'nstate',    'brake', 0),
            (   'nstate',    'brake', 1),
            (   'nstate',    'brake', 2),
            (   'nstate',   'thrust', 0),
            (   'nstate',   'thrust', 1),
            (   'nstate',   'thrus

In [17]:
dfs_episode1 = dfs_epi.copy().sort_index(axis=1)
dfs_episode1.set_index(('timestamp', '', 0), inplace=True)
dfs_episode1.index.name = 'timestamp'
idx = pd.IndexSlice
dfs_episode1

action                       nstate               
                             row0     row1     row2      brake               
                                0   1    0   1    0   1      0     1     2   
timestamp                                                                    
2023-06-09 16:17:41.649423     11  12   13  14   15  16   24.0  25.0  26.0  \
2023-06-09 17:17:41.649423     11  12   13  14   15  16   24.0  25.0  26.0   
2023-06-09 18:17:41.649423     11  12   13  14   15  16   24.0  25.0  26.0   
2023-06-09 19:17:41.649423     11  12   13  14   15  16   24.0  25.0  26.0   
2023-06-09 20:17:41.649423     11  12   13  14   15  16   24.0  25.0  26.0   

                                   ... state                              
                           thrust  ... brake           thrust             
                                0  ...     0    1    2      0    1    2   
timestamp                          ...                                    
2023-06-09 16:17:41.649423   21.0  ...   7.0  8.0  9.0    4.0  5.0  6.0  \
2023-06-09 17:17:41.649423   21.0  ...   7.0  8.0  9.0    4.0  5.0  6.0   
2023-06-09 18:17:41.649423   21.0  ...   7.0  8.0  9.0    4.0  5.0  6.0   
2023-06-09 19:17:41.649423   21.0  ...   7.0  8.0  9.0    4.0  5.0  6.0   
2023-06-09 20:17:41.649423   21.0  ...   7.0  8.0  9.0    4.0  5.0  6.0   

                                                    
                           velocity                 
                                  0    1    2    3  
timestamp                                           
2023-06-09 16:17:41.649423      0.0  1.0  2.0  3.0  
2023-06-09 17:17:41.649423      0.0  1.0  2.0  3.0  
2023-06-09 18:17:41.649423      0.0  1.0  2.0  3.0  
2023-06-09 19:17:41.649423      0.0  1.0  2.0  3.0  
2023-06-09 20:17:41.649423      0.0  1.0  2.0  3.0  

[5 rows x 27 columns]

In [18]:
dfs_episode1.loc[:, idx['action':'state', :, :]] = dfs_episode1.loc[
    :, idx['action':'state', :, :]
].astype('int')
dfs_episode1
dfs_episode1.dtypes
dfs_episode1.columns

action                       nstate                  
                             row0     row1     row2      brake         thrust   
                                0   1    0   1    0   1      0   1   2      0   
timestamp                                                                       
2023-06-09 16:17:41.649423     11  12   13  14   15  16     24  25  26     21  \
2023-06-09 17:17:41.649423     11  12   13  14   15  16     24  25  26     21   
2023-06-09 18:17:41.649423     11  12   13  14   15  16     24  25  26     21   
2023-06-09 19:17:41.649423     11  12   13  14   15  16     24  25  26     21   
2023-06-09 20:17:41.649423     11  12   13  14   15  16     24  25  26     21   

                            ... state                                       
                            ... brake       thrust       velocity           
                            ...     0  1  2      0  1  2        0  1  2  3  
timestamp                   ...                                             
2023-06-09 16:17:41.649423  ...     7  8  9      4  5  6        0  1  2  3  
2023-06-09 17:17:41.649423  ...     7  8  9      4  5  6        0  1  2  3  
2023-06-09 18:17:41.649423  ...     7  8  9      4  5  6        0  1  2  3  
2023-06-09 19:17:41.649423  ...     7  8  9      4  5  6        0  1  2  3  
2023-06-09 20:17:41.649423  ...     7  8  9      4  5  6        0  1  2  3  

[5 rows x 27 columns]

action  row0      0    object
                  1    object
        row1      0    object
                  1    object
        row2      0    object
                  1    object
nstate  brake     0    object
                  1    object
                  2    object
        thrust    0    object
                  1    object
                  2    object
        velocity  0    object
                  1    object
                  2    object
                  3    object
reward            0    object
state   brake     0    object
                  1    object
                  2    object
        thrust    0    object
                  1    object
                  2    object
        velocity  0    object
                  1    object
                  2    object
                  3    object
dtype: object

MultiIndex([('action',     'row0', 0),
            ('action',     'row0', 1),
            ('action',     'row1', 0),
            ('action',     'row1', 1),
            ('action',     'row2', 0),
            ('action',     'row2', 1),
            ('nstate',    'brake', 0),
            ('nstate',    'brake', 1),
            ('nstate',    'brake', 2),
            ('nstate',   'thrust', 0),
            ('nstate',   'thrust', 1),
            ('nstate',   'thrust', 2),
            ('nstate', 'velocity', 0),
            ('nstate', 'velocity', 1),
            ('nstate', 'velocity', 2),
            ('nstate', 'velocity', 3),
            ('reward',         '', 0),
            ( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'v

In [19]:
dfs_episode = dfs_epi.copy()
dfs_episode.set_index(('timestamp', '', 0), inplace=True)
dfs_episode.index.name = 'timestamp'
idx = pd.IndexSlice

# dfs_episode = dfs_episode.astype({idx['state', 'velocity', :]: 'int'})  # Error unhashable type: 'slice' for dictionary
# sorted_idx = **(idx['state', :, :]:'int')
# sorted_idx
dfs_episode['state'] = dfs_episode['state'].astype(
    'float'
)  # float16 not allowed in parquet
dfs_episode['action'] = dfs_episode['action'].astype('float')
dfs_episode['reward'] = dfs_episode['reward'].astype('float')
dfs_episode['nstate'] = dfs_episode['nstate'].astype('float')
dfs_episode
dfs_episode.dtypes
# dfs_episode.columns
# dfs_epi
# dfs_episode


state                                            
                           brake           thrust           velocity        
                               0    1    2      0    1    2        0    1   
timestamp                                                                   
2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0  \
2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   
2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   
2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   
2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0  5.0  6.0      0.0  1.0   

                                      ... nstate                            
                                      ...  brake             thrust         
                              2    3  ...      0     1     2      0     1   
timestamp                             ...                                   
2023-06-09 16:17:41.649423  2.0  3.0  ...   24.0  25.0  26.0   21.0  22.0  \
2023-06-09 17:17:41.649423  2.0  3.0  ...   24.0  25.0  26.0   21.0  22.0   
2023-06-09 18:17:41.649423  2.0  3.0  ...   24.0  25.0  26.0   21.0  22.0   
2023-06-09 19:17:41.649423  2.0  3.0  ...   24.0  25.0  26.0   21.0  22.0   
2023-06-09 20:17:41.649423  2.0  3.0  ...   24.0  25.0  26.0   21.0  22.0   

                                                             
                                 velocity                    
                               2        0     1     2     3  
timestamp                                                    
2023-06-09 16:17:41.649423  23.0     17.0  18.0  19.0  20.0  
2023-06-09 17:17:41.649423  23.0     17.0  18.0  19.0  20.0  
2023-06-09 18:17:41.649423  23.0     17.0  18.0  19.0  20.0  
2023-06-09 19:17:41.649423  23.0     17.0  18.0  19.0  20.0  
2023-06-09 20:17:41.649423  23.0     17.0  18.0  19.0  20.0  

[5 rows x 27 columns]

state   brake     0    float64
                  1    float64
                  2    float64
        thrust    0    float64
                  1    float64
                  2    float64
        velocity  0    float64
                  1    float64
                  2    float64
                  3    float64
reward            0    float64
action  row0      0    float64
                  1    float64
        row1      0    float64
                  1    float64
        row2      0    float64
                  1    float64
nstate  brake     0    float64
                  1    float64
                  2    float64
        thrust    0    float64
                  1    float64
                  2    float64
        velocity  0    float64
                  1    float64
                  2    float64
                  3    float64
dtype: object

## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [20]:
dfs_episode = pd.concat(
    [dfs_episode], keys=[drivers_by_id['wang-cheng'].pid], names=['driver']
)
dfs_episode = pd.concat(
    [dfs_episode], keys=[trucks_by_name['VB7'].vid], names=['vehicle']
)

In [21]:

dfs_episode.columns.set_names(
    ['tuple', 'measurement', 'step'], level=[0, 1, 2], inplace=True
)
dfs_episode
dfs_episode.dtypes


tuple                                         state                         
measurement                                   brake           thrust        
step                                              0    1    2      0    1   
vehicle driver     timestamp                                                
MP73    wang-cheng 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0  5.0  \
                   2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0  5.0   
                   2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0  5.0   
                   2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0  5.0   
                   2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0  5.0   

tuple                                                                        
measurement                                        velocity                  
step                                             2        0    1    2    3   
vehicle driver     timestamp                                                 
MP73    wang-cheng 2023-06-09 16:17:41.649423  6.0      0.0  1.0  2.0  3.0  \
                   2023-06-09 17:17:41.649423  6.0      0.0  1.0  2.0  3.0   
                   2023-06-09 18:17:41.649423  6.0      0.0  1.0  2.0  3.0   
                   2023-06-09 19:17:41.649423  6.0      0.0  1.0  2.0  3.0   
                   2023-06-09 20:17:41.649423  6.0      0.0  1.0  2.0  3.0   

tuple                                          ... nstate                      
measurement                                    ...  brake             thrust   
step                                           ...      0     1     2      0   
vehicle driver     timestamp                   ...                             
MP73    wang-cheng 2023-06-09 16:17:41.649423  ...   24.0  25.0  26.0   21.0  \
                   2023-06-09 17:17:41.649423  ...   24.0  25.0  26.0   21.0   
                   2023-06-09 18:17:41.649423  ...   24.0  25.0  26.0   21.0   
                   2023-06-09 19:17:41.649423  ...   24.0  25.0  26.0   21.0   
                   2023-06-09 20:17:41.649423  ...   24.0  25.0  26.0   21.0   

tuple                                                                      
measurement                                               velocity         
step                                              1     2        0     1   
vehicle driver     timestamp                                               
MP73    wang-cheng 2023-06-09 16:17:41.649423  22.0  23.0     17.0  18.0  \
                   2023-06-09 17:17:41.649423  22.0  23.0     17.0  18.0   
                   2023-06-09 18:17:41.649423  22.0  23.0     17.0  18.0   
                   2023-06-09 19:17:41.649423  22.0  23.0     17.0  18.0   
                   2023-06-09 20:17:41.649423  22.0  23.0     17.0  18.0   

tuple                                                      
measurement                                                
step                                              2     3  
vehicle driver     timestamp                               
MP73    wang-cheng 2023-06-09 16:17:41.649423  19.0  20.0  
                   2023-06-09 17:17:41.649423  19.0  20.0  
                   2023-06-09 18:17:41.649423  19.0  20.0  
                   2023-06-09 19:17:41.649423  19.0  20.0  
                   2023-06-09 20:17:41.649423  19.0  20.0  

[5 rows x 27 columns]

tuple   measurement  step
state   brake        0       float64
                     1       float64
                     2       float64
        thrust       0       float64
                     1       float64
                     2       float64
        velocity     0       float64
                     1       float64
                     2       float64
                     3       float64
reward               0       float64
action  row0         0       float64
                     1       float64
        row1         0       float64
                     1       float64
        row2         0       float64
                     1       float64
nstate  brake        0       float64
                     1       float64
                     2       float64
        thrust       0       float64
                     1       float64
                     2       float64
        velocity     0       float64
                     1       float64
                     2       float64
            

### Convert columns types to int and float16

In [22]:
# idx = pd.IndexSlice
# dfs_episode.loc[:, idx['state', :, :]] = dfs_episode.loc[:, idx['state', :, :]].astype(
#     'int'
# )
# dfs_episode.loc[:, idx['action', :, :]] = dfs_episode.loc[
#     :, idx['action', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['reward', :, :]] = dfs_episode.loc[
#     :, idx['reward', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['nstate', :, :]] = dfs_episode.loc[
#     :, idx['nstate', :, :]
# ].astype('float16')
#
# vel_1 = dfs_episode[[('state', 'velocity', 1)]]
# vel_1.dtypes
# vel_1.index
# vel_1.values
# # type(vel_1)
# vel_1.iloc[0]
# type(vel_1.iloc[0])
# type(vel_1.iloc[0].values[0])

## Add a level of index for episode

In [24]:
episodestart = ts - pd.Timedelta(1, 'h')
dfs_episode = pd.concat([dfs_episode], keys=[ts], names=['episodestart'])
dfs_episode.sort_index(inplace=True)
dfs_episode.index
dfs_episode.columns
dfs_episode

MultiIndex([('2023-06-09 15:17:41.649423', 'MP73', 'wang-cheng', ...),
            ('2023-06-09 15:17:41.649423', 'MP73', 'wang-cheng', ...),
            ('2023-06-09 15:17:41.649423', 'MP73', 'wang-cheng', ...),
            ('2023-06-09 15:17:41.649423', 'MP73', 'wang-cheng', ...),
            ('2023-06-09 15:17:41.649423', 'MP73', 'wang-cheng', ...)],
           names=['episodestart', 'vehicle', 'driver', 'timestamp'])

MultiIndex([( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'velocity', 2),
            ( 'state', 'velocity', 3),
            ('reward',         '', 0),
            ('action',     'row0', 0),
            ('action',     'row0', 1),
            ('action',     'row1', 0),
            ('action',     'row1', 1),
            ('action',     'row2', 0),
            ('action',     'row2', 1),
            ('nstate',    'brake', 0),
            ('nstate',    'brake', 1),
            ('nstate',    'brake', 2),
            ('nstate',   'thrust', 0),
            ('nstate',   'thrust', 1),
            ('nstate',   'thrust', 2),
            ('nstate', 'velocity', 0),
            ('nstate', 'velocity', 1),
            ('nstate', 'v

tuple                                                                    state   
measurement                                                              brake   
step                                                                         0   
episodestart               vehicle driver     timestamp                          
2023-06-09 15:17:41.649423 MP73    wang-cheng 2023-06-09 16:17:41.649423   7.0  \
                                              2023-06-09 17:17:41.649423   7.0   
                                              2023-06-09 18:17:41.649423   7.0   
                                              2023-06-09 19:17:41.649423   7.0   
                                              2023-06-09 20:17:41.649423   7.0   

tuple                                                                           
measurement                                                                     
step                                                                        1   
episodestart               vehicle driver     timestamp                         
2023-06-09 15:17:41.649423 MP73    wang-cheng 2023-06-09 16:17:41.649423  8.0  \
                                              2023-06-09 17:17:41.649423  8.0   
                                              2023-06-09 18:17:41.649423  8.0   
                                              2023-06-09 19:17:41.649423  8.0   
                                              2023-06-09 20:17:41.649423  8.0   

tuple                                                                           
measurement                                                                     
step                                                                        2   
episodestart               vehicle driver     timestamp                         
2023-06-09 15:17:41.649423 MP73    wang-cheng 2023-06-09 16:17:41.649423  9.0  \
                                              2023-06-09 17:17:41.649423  9.0   
                                              2023-06-09 18:17:41.649423  9.0   
                                              2023-06-09 19:17:41.649423  9.0   
                                              2023-06-09 20:17:41.649423  9.0   

tuple                                                                             
measurement                                                              thrust   
step                                                                          0   
episodestart               vehicle driver     timestamp                           
2023-06-09 15:17:41.649423 MP73    wang-cheng 2023-06-09 16:17:41.649423    4.0  \
                                              2023-06-09 17:17:41.649423    4.0   
                                              2023-06-09 18:17:41.649423    4.0   
                                              2023-06-09 19:17:41.649423    4.0   
                                              2023-06-09 20:17:41.649423    4.0   

tuple                                                                           
measurement                                                                     
step                                                                        1   
episodestart               vehicle driver     timestamp                         
2023-06-09 15:17:41.649423 MP73    wang-cheng 2023-06-09 16:17:41.649423  5.0  \
                                              2023-06-09 17:17:41.649423  5.0   
                                              2023-06-09 18:17:41.649423  5.0   
                                              2023-06-09 19:17:41.649423  5.0   
                                              2023-06-09 20:17:41.649423  5.0   

tuple                                                                           
measurement                                                                     
step                                                                        2   
episodestart               vehicle driver     timestamp                         

In [25]:

ts_new = pd.to_datetime((datetime.now()))
ts_new
episodestart = ts_new - pd.Timedelta(2, 'd')
episodestart

dfs_episode0 = dfs_episode.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level='episodestart')
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks[0].vid], [drivers[1].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode1 = dfs_episode.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, 'd')], level='episodestart'
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks[1].vid], [drivers[0].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode2 = dfs_episode.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, 'd')], level='episodestart'
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks[0].vid], [drivers[0].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode3 = dfs_episode.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, 'd')], level='episodestart'
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks[0].vid], [drivers[1].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode0



Timestamp('2023-06-09 15:22:43.124431')

Timestamp('2023-06-07 15:22:43.124431')

tuple                                                                      state   
measurement                                                                brake   
step                                                                           0   
episodestart               vehicle driver       timestamp                          
2023-06-07 15:22:43.124431 MP73    li-changlong 2023-06-09 16:17:41.649423   7.0  \
                                                2023-06-09 17:17:41.649423   7.0   
                                                2023-06-09 18:17:41.649423   7.0   
                                                2023-06-09 19:17:41.649423   7.0   
                                                2023-06-09 20:17:41.649423   7.0   

tuple                                                                             
measurement                                                                       
step                                                                          1   
episodestart               vehicle driver       timestamp                         
2023-06-07 15:22:43.124431 MP73    li-changlong 2023-06-09 16:17:41.649423  8.0  \
                                                2023-06-09 17:17:41.649423  8.0   
                                                2023-06-09 18:17:41.649423  8.0   
                                                2023-06-09 19:17:41.649423  8.0   
                                                2023-06-09 20:17:41.649423  8.0   

tuple                                                                             
measurement                                                                       
step                                                                          2   
episodestart               vehicle driver       timestamp                         
2023-06-07 15:22:43.124431 MP73    li-changlong 2023-06-09 16:17:41.649423  9.0  \
                                                2023-06-09 17:17:41.649423  9.0   
                                                2023-06-09 18:17:41.649423  9.0   
                                                2023-06-09 19:17:41.649423  9.0   
                                                2023-06-09 20:17:41.649423  9.0   

tuple                                                                               
measurement                                                                thrust   
step                                                                            0   
episodestart               vehicle driver       timestamp                           
2023-06-07 15:22:43.124431 MP73    li-changlong 2023-06-09 16:17:41.649423    4.0  \
                                                2023-06-09 17:17:41.649423    4.0   
                                                2023-06-09 18:17:41.649423    4.0   
                                                2023-06-09 19:17:41.649423    4.0   
                                                2023-06-09 20:17:41.649423    4.0   

tuple                                                                             
measurement                                                                       
step                                                                          1   
episodestart               vehicle driver       timestamp                         
2023-06-07 15:22:43.124431 MP73    li-changlong 2023-06-09 16:17:41.649423  5.0  \
                                                2023-06-09 17:17:41.649423  5.0   
                                                2023-06-09 18:17:41.649423  5.0   
                                                2023-06-09 19:17:41.649423  5.0   
                                                2023-06-09 20:17:41.649423  5.0   

tuple                                                                             
measurement                                                                       
step                                                               

In [ ]:
# dfs_episode_concat = pd.concat([dfs_episode, dfs_episode0], axis=0)
# display('dfs_episode_concat')
# dfs_episode_concat.index
# dfs_episode_concat.columns

In [26]:
episodes = [dfs_episode, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all[['state', 'action', 'reward', 'nstate']]
display('dfs_episode_all')
dfs_episode_all.index
dfs_episode_all.columns

dfs_episode_all
tuple                                                                      state   
measurement                                                                brake   
step                                                                           0   
vehicle driver       episodestart               timestamp                          
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  \
                                                2023-06-09 17:17:41.649423   7.0   
                                                2023-06-09 18:17:41.649423   7.0   
                                                2023-06-09 19:17:41.649423   7.0   
                                                2023-06-09 20:17:41.649423   7.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   7.0   
                                                2023-06-09 17:17:41.649423   7.0   
                                                2023-06-09 18:17:41.649423   7.0   
                                                2023-06-09 19:17:41.649423   7.0   
                                                2023-06-09 20:17:41.649423   7.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   7.0   
                                                2023-06-09 17:17:41.649423   7.0   
                                                2023-06-09 18:17:41.649423   7.0   
                                                2023-06-09 19:17:41.649423   7.0   
                                                2023-06-09 20:17:41.649423   7.0   
                     2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   7.0   
                                                2023-06-09 17:17:41.649423   7.0   
                                                2023-06-09 18:17:41.649423   7.0   
                                                2023-06-09 19:17:41.649423   7.0   
                                                2023-06-09 20:17:41.649423   7.0   
MP74    wang-cheng   2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   7.0   
                                                2023-06-09 17:17:41.649423   7.0   
                                                2023-06-09 18:17:41.649423   7.0   
                                                2023-06-09 19:17:41.649423   7.0   
                                                2023-06-09 20:17:41.649423   7.0   

tuple                                                                             
measurement                                                                       
step                                                                          1   
vehicle driver       episodestart               timestamp                         
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  8.0  \
                                                2023-06-09 17:17:41.649423  8.0   
                                                2023-06-09 18:17:41.649423  8.0   
                                                2023-06-09 19:17:41.649423  8.0   
                                                2023-06-09 20:17:41.649423  8.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  8.0   
                                                2023-06-09 17:17:41.649423  8.0   
                                                2023-06-09 18:17:41.649423  8.0   
                                                2023-06-09 19:17:41.649423  8.0   
                                                2023-06-09 20:17:41.649423  8.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  8.0   
                                                2023-06-09 17:17:41.649423  8.0   
                                                2023-06-09 18:17:41.649423  8.0   
                                                2023-06-09 19:17:41.649423  8.0   
                                                2023-

MultiIndex([('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15

MultiIndex([( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'velocity', 2),
            ( 'state', 'velocity', 3),
            ('action',     'row0', 0),
            ('action',     'row0', 1),
            ('action',     'row1', 0),
            ('action',     'row1', 1),
            ('action',     'row2', 0),
            ('action',     'row2', 1),
            ('reward',         '', 0),
            ('nstate',    'brake', 0),
            ('nstate',    'brake', 1),
            ('nstate',    'brake', 2),
            ('nstate',   'thrust', 0),
            ('nstate',   'thrust', 1),
            ('nstate',   'thrust', 2),
            ('nstate', 'velocity', 0),
            ('nstate', 'velocity', 1),
            ('nstate', 'v

## Make MultiIndex to Rows by adding levels of "Vehicle" and "Driver"

###  reset index to move vehicle and driver to columns, preprocessing for dask manipulation

In [27]:
dfs_episodes = dfs_episode_all.reset_index(
    level=['vehicle', 'driver']
)  # unstack vehicle and driver to columns in level 0 with default '' in level 1
# # dfs_episodes.columns
# # dfs_episode0.index = dfs_episode0.index.set_levels([[trucks[0].vid],[drivers[1].pid]], level=['vehicle', 'driver'], verify_integrity=False)
# old_columns = dfs_episodes.columns
# # old_columns[0]= (old_columns[0][0],0)
# new_columns_tuples = [(old_columns[0][0],0), (old_columns[1][0],0)] + old_columns[2:].to_list()
# prepend_columns = pd.MultiIndex.from_tuples(new_columns_tuples, names=old_columns.names)
# # prepend_columns

# dfs_episodes.columns = prepend_columns
dfs_episodes
# dfs_episodes.columns
# dfs_episodes.dtypes
# dfs_episode_all

tuple                                                 vehicle        driver   
measurement                                                                   
step                                                                          
episodestart               timestamp                                          
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423    MP73  li-changlong  \
                           2023-06-09 17:17:41.649423    MP73  li-changlong   
                           2023-06-09 18:17:41.649423    MP73  li-changlong   
                           2023-06-09 19:17:41.649423    MP73  li-changlong   
                           2023-06-09 20:17:41.649423    MP73  li-changlong   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423    MP73  li-changlong   
                           2023-06-09 17:17:41.649423    MP73  li-changlong   
                           2023-06-09 18:17:41.649423    MP73  li-changlong   
                           2023-06-09 19:17:41.649423    MP73  li-changlong   
                           2023-06-09 20:17:41.649423    MP73  li-changlong   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423    MP73    wang-cheng   
                           2023-06-09 17:17:41.649423    MP73    wang-cheng   
                           2023-06-09 18:17:41.649423    MP73    wang-cheng   
                           2023-06-09 19:17:41.649423    MP73    wang-cheng   
                           2023-06-09 20:17:41.649423    MP73    wang-cheng   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423    MP73    wang-cheng   
                           2023-06-09 17:17:41.649423    MP73    wang-cheng   
                           2023-06-09 18:17:41.649423    MP73    wang-cheng   
                           2023-06-09 19:17:41.649423    MP73    wang-cheng   
                           2023-06-09 20:17:41.649423    MP73    wang-cheng   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423    MP74    wang-cheng   
                           2023-06-09 17:17:41.649423    MP74    wang-cheng   
                           2023-06-09 18:17:41.649423    MP74    wang-cheng   
                           2023-06-09 19:17:41.649423    MP74    wang-cheng   
                           2023-06-09 20:17:41.649423    MP74    wang-cheng   

tuple                                                 state                    
measurement                                           brake           thrust   
step                                                      0    1    2      0   
episodestart               timestamp                                           
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0  \
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2

In [28]:
dfs_episodes.dtypes

tuple    measurement  step
vehicle                        object
driver                         object
state    brake        0       float64
                      1       float64
                      2       float64
         thrust       0       float64
                      1       float64
                      2       float64
         velocity     0       float64
                      1       float64
                      2       float64
                      3       float64
action   row0         0       float64
                      1       float64
         row1         0       float64
                      1       float64
         row2         0       float64
                      1       float64
reward                0       float64
nstate   brake        0       float64
                      1       float64
                      2       float64
         thrust       0       float64
                      1       float64
                      2       float64
         velocity     0

In [29]:
dfs_episodes.columns

MultiIndex([('vehicle',         '', ''),
            ( 'driver',         '', ''),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  'state', 'velocity',  0),
            (  'state', 'velocity',  1),
            (  'state', 'velocity',  2),
            (  'state', 'velocity',  3),
            ( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'reward',         '',  0),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'n

In [30]:

dfs_episodes['vehicle'] = dfs_episodes['vehicle'].astype("category")
dfs_episodes['driver'] = dfs_episodes['driver'].astype("category")
# dfs_episodes.dtypes
# dfs_episodes.columns
# dfs_episodes[['vehicle','driver']].dtypes
# dfs_episodes[['vehicle','driver']]
# dfs_episodes[['vehicle','driver']].iloc[:3].dtypes
# sliced = dfs_episodes[['vehicle']].iloc[:3].stack(level=[-1,-2])
# type(sliced)
# sliced.index
# sliced.values

dfs_episodes_sorted = dfs_episodes.sort_index(
    axis=1, level=[0, 1, 2], ascending=[True, True, True]
)
dfs_episodes_sorted.sort_index(axis=0, inplace=True)
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=1, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=0, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted.sort_index(axis=1,inplace=True)
dfs_episodes_sorted
dfs_episodes_sorted.columns

# dfs_episodes[['driver'], ['vehicle']] = dfs_episodes[['driver', 'vehicle']].apply(lambda x: x.cat.codes)

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.

MultiIndex([( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'driver',         '', ''),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  '

## dataframe slicing

In [31]:
ts
ep_start = ts - pd.Timedelta(9, 'D')
ep_start
# time_slice
episode_slice = pd.date_range(ep_start, periods=6, freq='D')
episode_slice

Timestamp('2023-06-09 15:17:41.649423')

Timestamp('2023-05-31 15:17:41.649423')

DatetimeIndex(['2023-05-31 15:17:41.649423', '2023-06-01 15:17:41.649423',
               '2023-06-02 15:17:41.649423', '2023-06-03 15:17:41.649423',
               '2023-06-04 15:17:41.649423', '2023-06-05 15:17:41.649423'],
              dtype='datetime64[ns]', freq='D')

In [32]:
timestamp_slice = pd.date_range(ts, periods=3, freq='H') + pd.Timedelta(10, 'min')
timestamp_slice

DatetimeIndex(['2023-06-09 15:27:41.649423', '2023-06-09 16:27:41.649423',
               '2023-06-09 17:27:41.649423'],
              dtype='datetime64[ns]', freq='H')

In [33]:
dfs_episodes_sorted

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.

In [34]:
dfs_episodes_sorted.index
dfs_episodes_sorted.columns

MultiIndex([('2023-06-02 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-04 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-04 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-04 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-04 15:22:43.1244

MultiIndex([( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'driver',         '', ''),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  '

In [35]:
# dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3]]
# dfs_selected_episodes = dfs_episodes_sorted.loc[trucks[0].vid, drivers[1].pid, episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
idx = pd.IndexSlice
dfs_selected_episodes = dfs_episodes_sorted.loc[
    idx[episode_slice[0] : episode_slice[5], timestamp_slice[0] : timestamp_slice[2]],
    idx['action':'driver'],
]
dfs_selected_episodes
# dfs_selected_episodes[['action', 'reward']]

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   

tuple                                                        driver  
measurement                                                          
step                                                                 
episodestart               timestamp                                 
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  li-changlong  
                           2023-06-09 17:17:41.649423  li-changlong  
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423    wang-cheng  
                           2023-06-09 17:17:41.649423    wang-cheng  
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423    wang-cheng  
                           2023-06-09 17:17:41.649423    wang-cheng

In [36]:
# dfs_selected_episodes_columns = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]][['action', 'reward']]
# dfs_selected_episodes_columns

In [37]:
# dfs_episode_all_sorted = dfs_episode_all.sort_index(axis=1)
# dfs_episode_all_sorted = dfs_episode_all_sorted.sort_index(axis=0)
# dfs_episode_all_sorted

In [38]:

# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(slice('VB6'), slice('wang-cheng'), slice(None), slice(None)),:]
# idx = pd.IndexSlice
# dfs_episode_all_sorted.loc[(idx['VB6'], idx['wang-cheng','zheng-longfei'], slice(None), slice(None)),:]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns = dfs_episodes_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns

In [39]:

ts_new = pd.to_datetime((datetime.now()))
episodestart = ts_new - pd.Timedelta(10, 'd') - pd.Timedelta(6, 'h')
episodestart
episode_slice = pd.date_range(episodestart, periods=20, freq='d') + pd.Timedelta(
    10, 'min'
)
episode_slice
ts
time_slice = pd.date_range(ts, periods=6, freq='h') - pd.Timedelta(10, 'min')
time_slice


Timestamp('2023-05-30 09:24:03.435656')

DatetimeIndex(['2023-05-30 09:34:03.435656', '2023-05-31 09:34:03.435656',
               '2023-06-01 09:34:03.435656', '2023-06-02 09:34:03.435656',
               '2023-06-03 09:34:03.435656', '2023-06-04 09:34:03.435656',
               '2023-06-05 09:34:03.435656', '2023-06-06 09:34:03.435656',
               '2023-06-07 09:34:03.435656', '2023-06-08 09:34:03.435656',
               '2023-06-09 09:34:03.435656', '2023-06-10 09:34:03.435656',
               '2023-06-11 09:34:03.435656', '2023-06-12 09:34:03.435656',
               '2023-06-13 09:34:03.435656', '2023-06-14 09:34:03.435656',
               '2023-06-15 09:34:03.435656', '2023-06-16 09:34:03.435656',
               '2023-06-17 09:34:03.435656', '2023-06-18 09:34:03.435656'],
              dtype='datetime64[ns]', freq='D')

Timestamp('2023-06-09 15:17:41.649423')

DatetimeIndex(['2023-06-09 15:07:41.649423', '2023-06-09 16:07:41.649423',
               '2023-06-09 17:07:41.649423', '2023-06-09 18:07:41.649423',
               '2023-06-09 19:07:41.649423', '2023-06-09 20:07:41.649423'],
              dtype='datetime64[ns]', freq='H')

In [40]:

episode_slice[4]: episode_slice[8]
idx[episode_slice[4] : episode_slice[8]]
idx[episode_slice[4] : episode_slice[8], slice(None)]

slice(Timestamp('2023-06-03 09:34:03.435656'), Timestamp('2023-06-07 09:34:03.435656'), None)

(slice(Timestamp('2023-06-03 09:34:03.435656'), Timestamp('2023-06-07 09:34:03.435656'), None),
 slice(None, None, None))

In [41]:
episode_slice[4]
episode_slice[11]
# dfs_selected_episodes = dfs_episodes_sorted.loc[idx['VB7'], slice(None), episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes
# epislice = idx(episode_slice[4]:episode_slice[11])

# dfs_selected_episodes = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[11],slice(None)], idx[dfs_episodes_sorted['driver']==drivers[0].pid]]
dfs_selected_episodes1 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers[0].pid
]
dfs_selected_episodes1
dfs_selected_episodes2 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers[0].pid
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes2
dfs_selected_episodes3 = dfs_episodes_sorted.loc[
    (dfs_episodes_sorted['driver'] == drivers[0].pid)
    & (dfs_episodes_sorted['vehicle'] == trucks[0].vid)
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes3
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[8],slice(None)]].loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)]
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid) & (idx[episode_slice[4]:episode_slice[8],slice(None)])]
# dfs_selected_episodes31
# dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes


Timestamp('2023-06-03 09:34:03.435656')

Timestamp('2023-06-10 09:34:03.435656')

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   

tuple                                                      driver nstate   
measurement                                                        brake   
step                                                                   0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng   24.0  \
                           2023-06-09 17:17:41.649423  wang-cheng   24.0   
                           2023-06-09 18:17:41.649423  wang-cheng   24.0   
                           2023-06-09 19:17:41.649423  wang-cheng   24.0   
                           2023-06-09 20:17:41.649423  wang-cheng   24.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng   24.0   
                           2023-06-09 17:17:41.649423  wang-cheng   24.0   
                           2023-06-09 18:17:41.649423  wang-cheng   24.0   
                           2023-06-09 19:17:41.649423  wang-cheng   24.0   
                           2023-06-09 20:17:41.649423  wang-cheng   24.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423  wang-cheng

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   

tuple                                                      driver nstate   
measurement                                                        brake   
step                                                                   0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng   24.0  \
                           2023-06-09 17:17:41.649423  wang-cheng   24.0   
                           2023-06-09 18:17:41.649423  wang-cheng   24.0   
                           2023-06-09 19:17:41.649423  wang-cheng   24.0   
                           2023-06-09 20:17:41.649423  wang-cheng   24.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng   24.0   
                           2023-06-09 17:17:41.649423  wang-cheng   24.0   
                           2023-06-09 18:17:41.649423  wang-cheng   24.0   
                           2023-06-09 19:17:41.649423  wang-cheng   24.0   
                           2023-06-09 20:17:41.649423  wang-cheng   24.0   

tuple                                                              ... state   
measurement                                                        ... brake   
step                                                      1     2  ...     1   
episodestart               timestamp                               ...         
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  25.0  26.0  ...   8.0  \
                           2023-06-09 17:17:41.649423  25.0  26.0  ...   8.0   
                           2023-06-09 18:17:41.649423  25.0  26.0  ...   8.0   
                           2023-06-09 19:17:41.649423  25.0  26.0  ...   8.0   
                           2023-06-09 20:17:41.649423  25.0  26.0  ...   8.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  25.0  26.0  ...   8.0   
                   

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   

tuple                                                      driver nstate   
measurement                                                        brake   
step                                                                   0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng   24.0  \
                           2023-06-09 17:17:41.649423  wang-cheng   24.0   
                           2023-06-09 18:17:41.649423  wang-cheng   24.0   
                           2023-06-09 19:17:41.649423  wang-cheng   24.0   
                           2023-06-09 20:17:41.649423  wang-cheng   24.0   

tuple                                                              ... state   
measurement                                                        ... brake   
step                                                      1     2  ...     1   
episodestart               timestamp                               ...         
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  25.0  26.0  ...   8.0  \
                           2023-06-09 17:17:41.649423  25.0  26.0  ...   8.0   
                           2023-06-09 18:17:41.649423  25.0  26.0  ...   8.0   
                           2023-06-09 19:17:41.649423  25.0  26.0  ...   8.0   
                           2023-06-09 20:17:41.649423  25.0  26.0  ...   8.0   

tuple                                                                         
measurement                                                thrust             
step                                                     2      0    1    2   
episodestart               timestamp                                          
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  9.0    4.0  5.0  6.0  \
                           2023-06-09 17:17:41.649423  9.0    4.0  5.0  6.0   
                           2023-06-09 18:17:41.649423  9.0    4.0  5.0  6.0   
                           2023-06-09 19:17:41.649423  9.0    4.0  5.0  6.0   
                           2023-06-09 20:17:41.649423  9.0    4.0  5.0  6.0   

tuple                                                                           
measurement                                           velocity                  
step                                                         0    1    2    3   
episodestart               timestamp                                            
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423      0.0  1.0  2.0  3.0  \
                           2023-06-09 17:17:41.649423      0.0  1.0  2.0  3.0   
                           2023-06-

In [42]:
idx[time_slice[0] : time_slice[3]]

slice(Timestamp('2023-06-09 15:07:41.649423'), Timestamp('2023-06-09 18:07:41.649423'), None)

In [43]:

dfs_selected_episodes3.loc[:, idx[:, :, 0:2]]
# dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[2]],:]
dfs_selected_episodes3.loc[idx[:, time_slice[0] : time_slice[3]], :]

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   

tuple                                                 nstate               
measurement                                            brake               
step                                                       0     1     2   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   24.0  25.0  26.0  \
                           2023-06-09 17:17:41.649423   24.0  25.0  26.0   
                           2023-06-09 18:17:41.649423   24.0  25.0  26.0   
                           2023-06-09 19:17:41.649423   24.0  25.0  26.0   
                           2023-06-09 20:17:41.649423   24.0  25.0  26.0   

tuple                                                         ... reward   
measurement                                           thrust  ...          
step                                                       0  ...      0   
episodestart               timestamp                          ...          
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   21.0  ...    NaN  \
                           2023-06-09 17:17:41.649423   21.0  ...    NaN   
                           2023-06-09 18:17:41.649423   21.0  ...    NaN   
                           2023-06-09 19:17:41.649423   21.0  ...    NaN   
                           2023-06-09 20:17:41.649423   21.0  ...    NaN   

tuple                                                 state                    
measurement                                           brake           thrust   
step                                                      0    1    2      0   
episodestart               timestamp                                           
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0  \
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   

tuple                                                                           
measurement                                                     velocity        
step                                                     1    2        0    1   
episodestart               timestamp                                            
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  5.0  6.0      0.0  1.0  \
                           2023-06-09 17:17:41.649423  5.0  6.0      0.0  1.0   
                           2023-06-09 18:17:41.649423  5.0  6.

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   

tuple                                                      driver nstate   
measurement                                                        brake   
step                                                                   0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng   24.0  \
                           2023-06-09 17:17:41.649423  wang-cheng   24.0   

tuple                                                              ... state   
measurement                                                        ... brake   
step                                                      1     2  ...     1   
episodestart               timestamp                               ...         
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  25.0  26.0  ...   8.0  \
                           2023-06-09 17:17:41.649423  25.0  26.0  ...   8.0   

tuple                                                                         
measurement                                                thrust             
step                                                     2      0    1    2   
episodestart               timestamp                                          
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  9.0    4.0  5.0  6.0  \
                           2023-06-09 17:17:41.649423  9.0    4.0  5.0  6.0   

tuple                                                                           
measurement                                           velocity                  
step                                                         0    1    2    3   
episodestart               timestamp                                            
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423      0.0  1.0  2.0  3.0  \
                           2023-06-09 17:17:41.649423      0.0  1.0  2.0  3.0   

tuple                                                 vehicle  
measurement                                                    
step                                                           
episodestart               timestamp                           
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423    MP73  
                           2023-06-09 17:17:41.649423    MP73  

[2 rows x 29 columns]

In [44]:
dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
time_slice[0]
time_slice[3]
idx[:, time_slice[0] : time_slice[3]]
dfs_selected_episodes4 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], :
]
# dfs_selected_episodes4 = dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[3],:],:]

dfs_selected_episodes4
dfs_selected_episodes41 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], 'action':'driver'
]
dfs_selected_episodes41
# dfs_selected_episodes4 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)].loc[idx[episode_slice[4]:episode_slice[8],time_slice[0]:time_slice[3]]]
# dfs_selected_episodes4

MultiIndex([('2023-06-03 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 20:17:41.649423')],
           names=['episodestart', 'timestamp'])

Timestamp('2023-06-09 15:07:41.649423')

Timestamp('2023-06-09 18:07:41.649423')

(slice(None, None, None),
 slice(Timestamp('2023-06-09 15:07:41.649423'), Timestamp('2023-06-09 18:07:41.649423'), None))

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   

tuple                                                      driver nstate   
measurement                                                        brake   
step                                                                   0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng   24.0  \
                           2023-06-09 17:17:41.649423  wang-cheng   24.0   

tuple                                                              ... state   
measurement                                                        ... brake   
step                                                      1     2  ...     1   
episodestart               timestamp                               ...         
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  25.0  26.0  ...   8.0  \
                           2023-06-09 17:17:41.649423  25.0  26.0  ...   8.0   

tuple                                                                         
measurement                                                thrust             
step                                                     2      0    1    2   
episodestart               timestamp                                          
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  9.0    4.0  5.0  6.0  \
                           2023-06-09 17:17:41.649423  9.0    4.0  5.0  6.0   

tuple                                                                           
measurement                                           velocity                  
step                                                         0    1    2    3   
episodestart               timestamp                                            
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423      0.0  1.0  2.0  3.0  \
                           2023-06-09 17:17:41.649423      0.0  1.0  2.0  3.0   

tuple                                                 vehicle  
measurement                                                    
step                                                           
episodestart               timestamp                           
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423    MP73  
                           2023-06-09 17:17:41.649423    MP73  

[2 rows x 29 columns]

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   

tuple                                                      driver  
measurement                                                        
step                                                               
episodestart               timestamp                               
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  wang-cheng  
                           2023-06-09 17:17:41.649423  wang-cheng

In [45]:

# dfs_selected_episodes = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns


In [46]:
dfs_episodes_sorted
# dfs_episode.loc[:,'state']
dfs_slice = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['vehicle'] == 'MP73', ['vehicle', 'driver', 'state', 'reward']
]
dfs_slice

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.

tuple                                                 vehicle        driver   
measurement                                                                   
step                                                                          
episodestart               timestamp                                          
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423    MP73  li-changlong  \
                           2023-06-09 17:17:41.649423    MP73  li-changlong   
                           2023-06-09 18:17:41.649423    MP73  li-changlong   
                           2023-06-09 19:17:41.649423    MP73  li-changlong   
                           2023-06-09 20:17:41.649423    MP73  li-changlong   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423    MP73    wang-cheng   
                           2023-06-09 17:17:41.649423    MP73    wang-cheng   
                           2023-06-09 18:17:41.649423    MP73    wang-cheng   
                           2023-06-09 19:17:41.649423    MP73    wang-cheng   
                           2023-06-09 20:17:41.649423    MP73    wang-cheng   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423    MP73  li-changlong   
                           2023-06-09 17:17:41.649423    MP73  li-changlong   
                           2023-06-09 18:17:41.649423    MP73  li-changlong   
                           2023-06-09 19:17:41.649423    MP73  li-changlong   
                           2023-06-09 20:17:41.649423    MP73  li-changlong   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423    MP73    wang-cheng   
                           2023-06-09 17:17:41.649423    MP73    wang-cheng   
                           2023-06-09 18:17:41.649423    MP73    wang-cheng   
                           2023-06-09 19:17:41.649423    MP73    wang-cheng   
                           2023-06-09 20:17:41.649423    MP73    wang-cheng   

tuple                                                 state                    
measurement                                           brake           thrust   
step                                                      0    1    2      0   
episodestart               timestamp                                           
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0  \
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   

tuple                                                                           
measurement                                                     velocity        
step                

# Saved episodes as parquet in multi-level folders "vehicle/driver/episode.parquet"

In [47]:
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
from datetime import datetime

import pyarrow as pa
import pyarrow.parquet as pq
import platform

print('Python: ', platform.python_version())
print('pandas: ', pd.__version__)
print('pyarrow: ', pa.__version__)


Python:  3.10.9
pandas:  2.0.1
pyarrow:  11.0.0


In [48]:
os.chdir('../../data')

In [49]:

os.getcwd()

'/home/n/devel/anting_remote/eos/data'

## create parquet files

In [50]:
dfs_episodes_sorted
dfs_episodes_sorted.index
dfs_episodes_sorted.columns
dfs_episodes_sorted.dtypes

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.

MultiIndex([('2023-06-02 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-04 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-04 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-04 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-04 15:22:43.1244

MultiIndex([( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'driver',         '', ''),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  '

tuple    measurement  step
action   row0         0        float64
                      1        float64
         row1         0        float64
                      1        float64
         row2         0        float64
                      1        float64
driver                        category
nstate   brake        0        float64
                      1        float64
                      2        float64
         thrust       0        float64
                      1        float64
                      2        float64
         velocity     0        float64
                      1        float64
                      2        float64
                      3        float64
reward                0        float64
state    brake        0        float64
                      1        float64
                      2        float64
         thrust       0        float64
                      1        float64
                      2        float64
         velocity     0        float6

In [51]:
# dfs_episode4dask = dfs_episode_all_sorted.swaplevel(0,-1, axis=0).swaplevel(1,3, axis=0).swaplevel(2,3, axis=0).unstack()
# dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# dfs_episode4dask
# # dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# # dfs_episode4dask.index
# # dfs_episode4dask.index
# # dfs_episode4dask.columns
# # dfs_episode4dask
# # dfs_episode_all_stack = dfs_episode_all.unstack(level=[-1,-2,-3,])
# # dfs_episode_all_stack

In [52]:
# dfs_episode4dask1 = dfs_episode4dask.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1)
# # dfs_episode4dask1
# dfs_episode4dask2 = dfs_episode4dask1.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1).swaplevel(-4,-5,axis=1)
# dfs_episode4dask2.sort_index(axis=1, inplace=True)
# dfs_episode4dask2


### decoding multiindex dataframe to flat index for Dask datframe

In [53]:
dfs_episodes_sorted.columns
cols = dfs_episodes_sorted.columns.to_flat_index()
cols

cols_str = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}'
    for x in cols
]
cols_str

MultiIndex([( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'driver',         '', ''),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  '

Index([    ('action', 'row0', 0),     ('action', 'row0', 1),
           ('action', 'row1', 0),     ('action', 'row1', 1),
           ('action', 'row2', 0),     ('action', 'row2', 1),
              ('driver', '', ''),    ('nstate', 'brake', 0),
          ('nstate', 'brake', 1),    ('nstate', 'brake', 2),
         ('nstate', 'thrust', 0),   ('nstate', 'thrust', 1),
         ('nstate', 'thrust', 2), ('nstate', 'velocity', 0),
       ('nstate', 'velocity', 1), ('nstate', 'velocity', 2),
       ('nstate', 'velocity', 3),         ('reward', '', 0),
           ('state', 'brake', 0),     ('state', 'brake', 1),
           ('state', 'brake', 2),    ('state', 'thrust', 0),
          ('state', 'thrust', 1),    ('state', 'thrust', 2),
        ('state', 'velocity', 0),  ('state', 'velocity', 1),
        ('state', 'velocity', 2),  ('state', 'velocity', 3),
             ('vehicle', '', '')],
      dtype='object')

['action_row0_0',
 'action_row0_1',
 'action_row1_0',
 'action_row1_1',
 'action_row2_0',
 'action_row2_1',
 'driver',
 'nstate_brake_0',
 'nstate_brake_1',
 'nstate_brake_2',
 'nstate_thrust_0',
 'nstate_thrust_1',
 'nstate_thrust_2',
 'nstate_velocity_0',
 'nstate_velocity_1',
 'nstate_velocity_2',
 'nstate_velocity_3',
 'reward__0',
 'state_brake_0',
 'state_brake_1',
 'state_brake_2',
 'state_thrust_0',
 'state_thrust_1',
 'state_thrust_2',
 'state_velocity_0',
 'state_velocity_1',
 'state_velocity_2',
 'state_velocity_3',
 'vehicle']

In [54]:
dfs_episodes_sorted_flat_col = dfs_episodes_sorted.copy()
cols = dfs_episodes_sorted.columns.to_flat_index()
dfs_episodes_sorted_flat_col.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'  # !!! Very Important!!! dunder means the next of the next level! but str.splitt('_'), you get the '' in between for free!
    if (x[2] != '')
    else f'{x[0]}'
    for x in cols
]
dfs_episodes_sorted_flat_ind = dfs_episodes_sorted.copy()
idx = dfs_episodes_sorted.index.to_flat_index()
dfs_episodes_sorted_flat_ind.index = [
    f'{x[0]}_{x[1]}' if x[1] != '' else f'x{0}' for x in idx
]
# dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat = dfs_episodes_sorted.copy()
dfs_episodes_sorted_flat.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}'
    for x in cols
]
dfs_episodes_sorted_flat.index = [
    f'{idx[0]}_{idx[1]}' if idx[1] != '' else f'{idx[0]}'
    for idx in dfs_episodes_sorted.index.to_flat_index()
]
dfs_episodes_sorted_flat



,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,driver,nstate_brake_0,nstate_brake_1,nstate_brake_2,...,state_brake_1,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73


In [55]:
dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat_col
dfs_episodes_sorted_flat
dfs_episodes_sorted_flat.dtypes

tuple                                              action                     
measurement                                          row0        row1         
step                                                    0     1     0     1   
2023-06-02 15:22:43.124431_2023-06-09 16:17:41....   11.0  12.0  13.0  14.0  \
2023-06-02 15:22:43.124431_2023-06-09 17:17:41....   11.0  12.0  13.0  14.0   
2023-06-02 15:22:43.124431_2023-06-09 18:17:41....   11.0  12.0  13.0  14.0   
2023-06-02 15:22:43.124431_2023-06-09 19:17:41....   11.0  12.0  13.0  14.0   
2023-06-02 15:22:43.124431_2023-06-09 20:17:41....   11.0  12.0  13.0  14.0   
2023-06-03 15:22:43.124431_2023-06-09 16:17:41....   11.0  12.0  13.0  14.0   
2023-06-03 15:22:43.124431_2023-06-09 17:17:41....   11.0  12.0  13.0  14.0   
2023-06-03 15:22:43.124431_2023-06-09 18:17:41....   11.0  12.0  13.0  14.0   
2023-06-03 15:22:43.124431_2023-06-09 19:17:41....   11.0  12.0  13.0  14.0   
2023-06-03 15:22:43.124431_2023-06-09 20:17:41....   11.0  12.0  13.0  14.0   
2023-06-04 15:22:43.124431_2023-06-09 16:17:41....   11.0  12.0  13.0  14.0   
2023-06-04 15:22:43.124431_2023-06-09 17:17:41....   11.0  12.0  13.0  14.0   
2023-06-04 15:22:43.124431_2023-06-09 18:17:41....   11.0  12.0  13.0  14.0   
2023-06-04 15:22:43.124431_2023-06-09 19:17:41....   11.0  12.0  13.0  14.0   
2023-06-04 15:22:43.124431_2023-06-09 20:17:41....   11.0  12.0  13.0  14.0   
2023-06-07 15:22:43.124431_2023-06-09 16:17:41....   11.0  12.0  13.0  14.0   
2023-06-07 15:22:43.124431_2023-06-09 17:17:41....   11.0  12.0  13.0  14.0   
2023-06-07 15:22:43.124431_2023-06-09 18:17:41....   11.0  12.0  13.0  14.0   
2023-06-07 15:22:43.124431_2023-06-09 19:17:41....   11.0  12.0  13.0  14.0   
2023-06-07 15:22:43.124431_2023-06-09 20:17:41....   11.0  12.0  13.0  14.0   
2023-06-09 15:17:41.649423_2023-06-09 16:17:41....   11.0  12.0  13.0  14.0   
2023-06-09 15:17:41.649423_2023-06-09 17:17:41....   11.0  12.0  13.0  14.0   
2023-06-09 15:17:41.649423_2023-06-09 18:17:41....   11.0  12.0  13.0  14.0   
2023-06-09 15:17:41.649423_2023-06-09 19:17:41....   11.0  12.0  13.0  14.0   
2023-06-09 15:17:41.649423_2023-06-09 20:17:41....   11.0  12.0  13.0  14.0   

tuple                                                                 driver   
measurement                                         row2                       
step                                                   0     1                 
2023-06-02 15:22:43.124431_2023-06-09 16:17:41....  15.0  16.0  li-changlong  \
2023-06-02 15:22:43.124431_2023-06-09 17:17:41....  15.0  16.0  li-changlong   
2023-06-02 15:22:43.124431_2023-06-09 18:17:41....  15.0  16.0  li-changlong   
2023-06-02 15:22:43.124431_2023-06-09 19:17:41....  15.0  16.0  li-changlong   
2023-06-02 15:22:43.124431_2023-06-09 20:17:41....  15.0  16.0  li-changlong   
2023-06-03 15:22:43.124431_2023-06-09 16:17:41....  15.0  16.0    wang-cheng   
2023-06-03 15:22:43.124431_2023-06-09 17:17:41....  15.0  16.0    wang-cheng   
2023-06-03 15:22:43.124431_2023-06-09 18:17:41....  15.0  16.0    wang-cheng   
2023-06-03 15:22:43.124431_2023-06-09 19:17:41....  15.0  16.0    wang-cheng   
2023-06-03 15:22:43.124431_2023-06-09 20:17:41....  15.0  16.0    wang-cheng   
2023-06-04 15:22:43.124431_2023-06-09 16:17:41....  15.0  16.0    wang-cheng   
2023-06-04 15:22:43.124431_2023-06-09 17:17:41....  15.0  16.0    wang-cheng   
2023-06-04 15:22:43.124431_2023-06-09 18:17:41....  15.0  16.0    wang-cheng   
2023-06-04 15:22:43.124431_2023-06-09 19:17:41....  15.0  16.0    wang-cheng   
2023-06-04 15:22:43.124431_2023-06-09 20:17:41....  15.0  16.0    wang-cheng   
2023-06-07 15:22:43.124431_2023-06-09 16:17:41....  15.0  16.0  li-changlong   
2023-06-07 15:22:43.124431_2023-06-09 17:17:41....  15.0  16.0  li-changlong   
2023-06-07 15:22:43.124431_2023-06-09 18:17:41....  15.0  16.0  li-changlong   
2023-06-07 15:22:43.124431_2023-06-09 19:17:41....  15.0  16.0  li-changlong   
2023-06-07 15:22:43.124431_

action_row0_0   
episodestart               timestamp                                   
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423           11.0  \
                           2023-06-09 17:17:41.649423           11.0   
                           2023-06-09 18:17:41.649423           11.0   
                           2023-06-09 19:17:41.649423           11.0   
                           2023-06-09 20:17:41.649423           11.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423           11.0   
                           2023-06-09 17:17:41.649423           11.0   
                           2023-06-09 18:17:41.649423           11.0   
                           2023-06-09 19:17:41.649423           11.0   
                           2023-06-09 20:17:41.649423           11.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423           11.0   
                           2023-06-09 17:17:41.649423           11.0   
                           2023-06-09 18:17:41.649423           11.0   
                           2023-06-09 19:17:41.649423           11.0   
                           2023-06-09 20:17:41.649423           11.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423           11.0   
                           2023-06-09 17:17:41.649423           11.0   
                           2023-06-09 18:17:41.649423           11.0   
                           2023-06-09 19:17:41.649423           11.0   
                           2023-06-09 20:17:41.649423           11.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423           11.0   
                           2023-06-09 17:17:41.649423           11.0   
                           2023-06-09 18:17:41.649423           11.0   
                           2023-06-09 19:17:41.649423           11.0   
                           2023-06-09 20:17:41.649423           11.0   

                                                       action_row0_1   
episodestart               timestamp                                   
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423           12.0  \
                           2023-06-09 17:17:41.649423           12.0   
                           2023-06-09 18:17:41.649423           12.0   
                           2023-06-09 19:17:41.649423           12.0   
                           2023-06-09 20:17:41.649423           12.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423           12.0   
                           2023-06-09 17:17:41.649423           12.0   
                           2023-06-09 18:17:41.649423           12.0   
                           2023-06-09 19:17:41.649423           12.0   
                           2023-06-09 20:17:41.649423           12.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423           12.0   
                           2023-06-09 17:17:41.649423           12.0   
                           2023-06-09 18:17:41.649423           12.0   
                           2023-06-09 19:17:41.649423           12.0   
                           2023-06-09 20:17:41.649423           12.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423           12.0   
                           2023-06-09 17:17:41.649423           12.0   
                           2023-06-09 18:17:41.649423           12.0   
                           2023-06-09 19:17:41.649423           12.0   
                           2023-06-09 20:17:41.649423           12.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423           12.0   
                           2023-06-09 17:17:41.649423           12.0   
                           2023-06-09 18:17:41.649423           12.0   
                           2023-06-09 19:17:41.649423           12.0   
                           2023-06-09 20:17:41.649423           12.0   

                                                       action_row1_0   
episodestart               timestamp                                   
2023-06-02 15:22:43.1

,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,driver,nstate_brake_0,nstate_brake_1,nstate_brake_2,...,state_brake_1,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,li-changlong,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73
2023-06-03 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,wang-cheng,24.0,25.0,26.0,...,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73


action_row0_0         float64
action_row0_1         float64
action_row1_0         float64
action_row1_1         float64
action_row2_0         float64
action_row2_1         float64
driver               category
nstate_brake_0        float64
nstate_brake_1        float64
nstate_brake_2        float64
nstate_thrust_0       float64
nstate_thrust_1       float64
nstate_thrust_2       float64
nstate_velocity_0     float64
nstate_velocity_1     float64
nstate_velocity_2     float64
nstate_velocity_3     float64
reward__0             float64
state_brake_0         float64
state_brake_1         float64
state_brake_2         float64
state_thrust_0        float64
state_thrust_1        float64
state_thrust_2        float64
state_velocity_0      float64
state_velocity_1      float64
state_velocity_2      float64
state_velocity_3      float64
vehicle              category
dtype: object

In [56]:
dfs_episodes_sorted_flat['action_row0_0'].iloc[0:2].values.dtype

dtype('float64')

In [57]:
try:
    ddf_episodes = dd.from_pandas(dfs_episodes_sorted_flat, npartitions=1)
except Exception as e:
    print(e)

try:
    ddf_episodes.index
    ddf_episodes.columns
except Exception as e:
    print(e)

ddf_episodes

Dask Index Structure:
npartitions=1
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423    object
2023-06-09 15:17:41.649423_2023-06-09 20:17:41.649423       ...
dtype: object
Dask Name: from_pandas-index, 2 graph layers

Index(['action_row0_0', 'action_row0_1', 'action_row1_0', 'action_row1_1',
       'action_row2_0', 'action_row2_1', 'driver', 'nstate_brake_0',
       'nstate_brake_1', 'nstate_brake_2', 'nstate_thrust_0',
       'nstate_thrust_1', 'nstate_thrust_2', 'nstate_velocity_0',
       'nstate_velocity_1', 'nstate_velocity_2', 'nstate_velocity_3',
       'reward__0', 'state_brake_0', 'state_brake_1', 'state_brake_2',
       'state_thrust_0', 'state_thrust_1', 'state_thrust_2',
       'state_velocity_0', 'state_velocity_1', 'state_velocity_2',
       'state_velocity_3', 'vehicle'],
      dtype='object')

,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,driver,nstate_brake_0,nstate_brake_1,nstate_brake_2,nstate_thrust_0,nstate_thrust_1,nstate_thrust_2,nstate_velocity_0,nstate_velocity_1,nstate_velocity_2,nstate_velocity_3,reward__0,state_brake_0,state_brake_1,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,float64,float64,float64,float64,float64,float64,category[known],float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,category[known]
2023-06-09 15:17:41.649423_2023-06-09 20:17:41.649423,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [58]:

custom_metadata = {
    b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}
custom_metadata

{b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'}

In [59]:
try:
    with ProgressBar():
        ddf_episodes.to_parquet(
            'eos_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle', 'driver'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

[########################################] | 100% Completed | 101.42 ms


### Read saved parquet file

In [60]:
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.head(5)
ddf.dtypes
ddf.npartitions
ddf.divisions
ddf._meta

,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,nstate_brake_0,nstate_brake_1,nstate_brake_2,nstate_thrust_0,...,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle,driver
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong


action_row0_0         float64
action_row0_1         float64
action_row1_0         float64
action_row1_1         float64
action_row2_0         float64
action_row2_1         float64
nstate_brake_0        float64
nstate_brake_1        float64
nstate_brake_2        float64
nstate_thrust_0       float64
nstate_thrust_1       float64
nstate_thrust_2       float64
nstate_velocity_0     float64
nstate_velocity_1     float64
nstate_velocity_2     float64
nstate_velocity_3     float64
reward__0             float64
state_brake_0         float64
state_brake_1         float64
state_brake_2         float64
state_thrust_0        float64
state_thrust_1        float64
state_thrust_2        float64
state_velocity_0      float64
state_velocity_1      float64
state_velocity_2      float64
state_velocity_3      float64
vehicle              category
driver               category
dtype: object

3

(None, None, None, None)

,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,nstate_brake_0,nstate_brake_1,nstate_brake_2,nstate_thrust_0,...,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle,driver


### Get customized metadata with pyarrow

In [61]:

import pyarrow as pa
import pyarrow.parquet as pq
import json

# print('pyarrow: ', pa.__version__)

# table = pq.read_table('ddf_episodes')
# table_meta = table.schema.metadata[b'eos']
# print(table_meta)
# # custom_metadata_info = json.loads(table.schema.metadata[b'eos'])
# # print(custom_metadata_info)
# try:
#     custom_meta_info = json.loads(table_meta)
# except Exception as e:
#     print(f'Exception: {e}')

# print(custom_meta_info)

# # Print formatted output of the dichtionary custom_meta_info:
# for key, val in custom_meta_info.items():
#     print('{:15}: {}'.format(key, val))

# ALTERNATE:

table_meta = pq.read_metadata('eos_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = table_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

# table_meta = pq.read_metadata('ddf_episodes/_metadata')
# # print(table_meta)
# # print(table_meta.metadata[b'eos'])
# custom_meta_info = table_meta.metadata[b'eos']
# custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

length: [3, 600, 5]
timezeone: sh
units: ['kph', 'pct', 'pct']
data_propensity: [50, 1, 4]
Dataset Description: MP vehicle from TBox


### encoding flat_index of dask dataframe to multiindex

In [62]:
df_episodes_read = ddf.compute()
df_episodes_read

,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,nstate_brake_0,nstate_brake_1,nstate_brake_2,nstate_thrust_0,...,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle,driver
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong


In [63]:
multi_cols = [tuple(col.split('_')) for col in df_episodes_read.columns]
multi_cols
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_cols)
# multi_idx
multi_idx.names = ['tuple', 'measurement', 'step']
driver_vehicle = [idx for idx in multi_idx if idx[0] == 'driver' or idx[0] == 'vehicle']
driver_vehicle
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_idx

[('action', 'row0', '0'),
 ('action', 'row0', '1'),
 ('action', 'row1', '0'),
 ('action', 'row1', '1'),
 ('action', 'row2', '0'),
 ('action', 'row2', '1'),
 ('nstate', 'brake', '0'),
 ('nstate', 'brake', '1'),
 ('nstate', 'brake', '2'),
 ('nstate', 'thrust', '0'),
 ('nstate', 'thrust', '1'),
 ('nstate', 'thrust', '2'),
 ('nstate', 'velocity', '0'),
 ('nstate', 'velocity', '1'),
 ('nstate', 'velocity', '2'),
 ('nstate', 'velocity', '3'),
 ('reward', '', '0'),
 ('state', 'brake', '0'),
 ('state', 'brake', '1'),
 ('state', 'brake', '2'),
 ('state', 'thrust', '0'),
 ('state', 'thrust', '1'),
 ('state', 'thrust', '2'),
 ('state', 'velocity', '0'),
 ('state', 'velocity', '1'),
 ('state', 'velocity', '2'),
 ('state', 'velocity', '3'),
 ('vehicle',),
 ('driver',)]

[('vehicle', nan, nan), ('driver', nan, nan)]

In [64]:
# pd.Series([1, np.nan,2,None])
# pd.Series([1,np.nan,2,None,pd.NA])

In [65]:
multi_tpl = [tuple(col.split('_')) for col in df_episodes_read.columns]
# multi_tpl
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_idx
# multi_col
i1 = multi_col.get_level_values(0)
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
]  # convert index of level 2 type to int and '' if NA
i1
# i2 = multi_col.get_level_values(1).fillna('') # must be null string instead of the default pd.NA or np.nan
# i2 = [idx if isinstance(idx, int) else '' for idx in i2]
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i2
# i2 = [idx if isinstance(idx, str) else '' for idx in i2]
# i2.astype('int')
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA
i3

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
multi_col


# i2[-1]
# i2.dropna()
# i2
# l2 = ['' if i==nan else i  for i in i2]
# l2
# i2= i2[:-2].astype('int')
# i2 = i2.append(pd.Index(['','']))
# l2 = [''] + list(l2.values[:-1])
# print(l2)
# arrays = [ for i2 in l2 for i1 in l1]
# l2 = l2[:-2].astype('int') + ['', '']
# multi_col.set_levels(ll, level=['tuple','step'])
# driver_vehicle = [idx for idx in multi_idx if idx[0]=='driver' or idx[0]=='vehicle']
# multi_tpl =  [(col[0],)  if col[0]=='driver' or col[0]=='vehicle' else (col[0],int(col[1])) for col in multi_col]
# multi_idx1 = pd.MultiIndex.from_tuples(multi_idx1)
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_col

['action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'reward',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'vehicle',
 'driver']

['row0',
 'row0',
 'row1',
 'row1',
 'row2',
 'row2',
 'brake',
 'brake',
 'brake',
 'thrust',
 'thrust',
 'thrust',
 'velocity',
 'velocity',
 'velocity',
 'velocity',
 '',
 'brake',
 'brake',
 'brake',
 'thrust',
 'thrust',
 'thrust',
 'velocity',
 'velocity',
 'velocity',
 'velocity',
 '',
 '']

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 3,
 0,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 3,
 '',
 '']

MultiIndex([( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  'state', 'velocity',  0),
            (  '

In [66]:
multi_idx = [tuple(idx.split('_')) for idx in df_episodes_read.index]
# multi_idx
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
multi_idx
# len(multi_idx)

MultiIndex([('2023-06-02 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-03 15:22:43.1244

In [67]:
df_episodes_read_multi = df_episodes_read.copy()
df_episodes_read_multi.columns = multi_col
df_episodes_read_multi.index = multi_idx
df_episodes_read_multi
df_episodes_read_multi.columns
df_episodes_read_multi.index
df_episodes_read_multi.dtypes

tuple                                                 action               
measurement                                             row0        row1   
step                                                       0     1     0   
episodestart               timestamp                                       
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0  \
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   
2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 17:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 18:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 19:17:41.649423   11.0  12.0  13.0   
                           2023-06-09 20:17:41.649423   11.0  12.0  13.0   

tuple                                                                     
measurement                                                  row2         
step                                                      1     0     1   
episodestart               timestamp                                      
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0  \
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.0   
2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 17:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 18:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 19:17:41.649423  14.0  15.0  16.0   
                           2023-06-09 20:17:41.649423  14.0  15.0  16.

MultiIndex([( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  'state', 'velocity',  0),
            (  '

MultiIndex([('2023-06-02 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-03 15:22:43.1244

tuple    measurement  step
action   row0         0        float64
                      1        float64
         row1         0        float64
                      1        float64
         row2         0        float64
                      1        float64
nstate   brake        0        float64
                      1        float64
                      2        float64
         thrust       0        float64
                      1        float64
                      2        float64
         velocity     0        float64
                      1        float64
                      2        float64
                      3        float64
reward                0        float64
state    brake        0        float64
                      1        float64
                      2        float64
         thrust       0        float64
                      1        float64
                      2        float64
         velocity     0        float64
                      1        float6

### recover deep multiindex from columns ['vehicle','driver']

In [68]:
df_episodes_read_multi_setindex = df_episodes_read_multi.set_index(
    ['vehicle', 'driver', df_episodes_read_multi.index]
)
df_episodes_read_multi_setindex.index
df_episodes_read_multi_setindex.columns
df_episodes_read_multi_setindex

MultiIndex([('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15

MultiIndex([('action',     'row0', 0),
            ('action',     'row0', 1),
            ('action',     'row1', 0),
            ('action',     'row1', 1),
            ('action',     'row2', 0),
            ('action',     'row2', 1),
            ('nstate',    'brake', 0),
            ('nstate',    'brake', 1),
            ('nstate',    'brake', 2),
            ('nstate',   'thrust', 0),
            ('nstate',   'thrust', 1),
            ('nstate',   'thrust', 2),
            ('nstate', 'velocity', 0),
            ('nstate', 'velocity', 1),
            ('nstate', 'velocity', 2),
            ('nstate', 'velocity', 3),
            ('reward',         '', 0),
            ( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'v

tuple                                                                      action   
measurement                                                                  row0   
step                                                                            0   
vehicle driver       episodestart               timestamp                           
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  \
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
MP74    wang-cheng   2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   

tuple                                                                              
measurement                                                                        
step                                                                           1   
vehicle driver       episodestart               timestamp                          
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  12.0  \
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
                                                2023-06-09 20:17:41.649423  12.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  12.0   
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
                                                2023-06-09 20:17:41.649423  12.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423  12.0   
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
                      

## append new episode to the specific parquet file

In [69]:
dfs_episode4 = dfs_episode.copy()
dfs_episode4.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(8, 'd')], level='episodestart'
)
dfs_episode4.index = dfs_episode3.index.set_levels(
    [[trucks[1].vid], [drivers[1].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode4 = dfs_episode4.swaplevel(0, 1, axis=0).swaplevel(1, 2, axis=0)
dfs_episode4.columns

# dfs_episode5 = dfs_episode.copy()
# dfs_episode5.index = dfs_episode3.index.set_levels([episodestart-pd.Timedelta(9,'d')], level='episode')
df_episodes_read_multi_setindex.columns

MultiIndex([( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'velocity', 2),
            ( 'state', 'velocity', 3),
            ('reward',         '', 0),
            ('action',     'row0', 0),
            ('action',     'row0', 1),
            ('action',     'row1', 0),
            ('action',     'row1', 1),
            ('action',     'row2', 0),
            ('action',     'row2', 1),
            ('nstate',    'brake', 0),
            ('nstate',    'brake', 1),
            ('nstate',    'brake', 2),
            ('nstate',   'thrust', 0),
            ('nstate',   'thrust', 1),
            ('nstate',   'thrust', 2),
            ('nstate', 'velocity', 0),
            ('nstate', 'velocity', 1),
            ('nstate', 'v

MultiIndex([('action',     'row0', 0),
            ('action',     'row0', 1),
            ('action',     'row1', 0),
            ('action',     'row1', 1),
            ('action',     'row2', 0),
            ('action',     'row2', 1),
            ('nstate',    'brake', 0),
            ('nstate',    'brake', 1),
            ('nstate',    'brake', 2),
            ('nstate',   'thrust', 0),
            ('nstate',   'thrust', 1),
            ('nstate',   'thrust', 2),
            ('nstate', 'velocity', 0),
            ('nstate', 'velocity', 1),
            ('nstate', 'velocity', 2),
            ('nstate', 'velocity', 3),
            ('reward',         '', 0),
            ( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'v

In [70]:
dfs_episodes_new = pd.concat([df_episodes_read_multi_setindex, dfs_episode4], axis=0)
dfs_episodes_new
# dfs_episodes_new.columns
# dfs_episodes_new.index

tuple                                                                      action   
measurement                                                                  row0   
step                                                                            0   
vehicle driver       episodestart               timestamp                           
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  \
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
MP74    wang-cheng   2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   

tuple                                                                              
measurement                                                                        
step                                                                           1   
vehicle driver       episodestart               timestamp                          
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  12.0  \
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
                                                2023-06-09 20:17:41.649423  12.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  12.0   
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
                 

#### save new dataframe back to parquet

In [71]:

os.getcwd()

'/home/n/devel/anting_remote/eos/data'

In [78]:
dfs_episodes_new_flat = dfs_episodes_new.copy()
dfs_episodes_new_flat = dfs_episodes_new_flat.reset_index(level=['vehicle', 'driver'])
dfs_episodes_new_flat.columns

MultiIndex([('vehicle',         '', ''),
            ( 'driver',         '', ''),
            ( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  '

In [79]:
dfs_episodes_new_flat.index = [
    f'{idx[0]}_{idx[1]}' if idx[1] != '' else f'{idx[0]}'
    for idx in dfs_episodes_new_flat.index.to_flat_index()
]
dfs_episodes_new_flat.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}'
    for x in dfs_episodes_new_flat.columns.to_flat_index()
]
# dfs_episodes_new_flat['vehicle'] = dfs_episodes_new_flat['vehicle'].astype("category")
# dfs_episodes_new_flat['driver'] = dfs_episodes_new_flat['driver'].astype("category")

dfs_episodes_new_flat.index.name = 'episodestart_timestamp'
dfs_episodes_new_flat.index
dfs_episodes_new_flat.dtypes
dfs_episodes_new_flat.columns
dfs_episodes_new_flat


Index(['2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423',
       '2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423',
       '2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423',
       '2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423',
       '2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423',
       '2023-06-07 15:22:43.124431_2023-06-09 16:17:41.649423',
       '2023-06-07 15:22:43.124431_2023-06-09 17:17:41.649423',
       '2023-06-07 15:22:43.124431_2023-06-09 18:17:41.649423',
       '2023-06-07 15:22:43.124431_2023-06-09 19:17:41.649423',
       '2023-06-07 15:22:43.124431_2023-06-09 20:17:41.649423',
       '2023-06-03 15:22:43.124431_2023-06-09 16:17:41.649423',
       '2023-06-03 15:22:43.124431_2023-06-09 17:17:41.649423',
       '2023-06-03 15:22:43.124431_2023-06-09 18:17:41.649423',
       '2023-06-03 15:22:43.124431_2023-06-09 19:17:41.649423',
       '2023-06-03 15:22:43.124431_2023-06-09 20:17:41.649423',
       '2023-06-09 15:17:41.649423_2023-

vehicle               object
driver                object
action_row0_0        float64
action_row0_1        float64
action_row1_0        float64
action_row1_1        float64
action_row2_0        float64
action_row2_1        float64
nstate_brake_0       float64
nstate_brake_1       float64
nstate_brake_2       float64
nstate_thrust_0      float64
nstate_thrust_1      float64
nstate_thrust_2      float64
nstate_velocity_0    float64
nstate_velocity_1    float64
nstate_velocity_2    float64
nstate_velocity_3    float64
reward__0            float64
state_brake_0        float64
state_brake_1        float64
state_brake_2        float64
state_thrust_0       float64
state_thrust_1       float64
state_thrust_2       float64
state_velocity_0     float64
state_velocity_1     float64
state_velocity_2     float64
state_velocity_3     float64
dtype: object

Index(['vehicle', 'driver', 'action_row0_0', 'action_row0_1', 'action_row1_0',
       'action_row1_1', 'action_row2_0', 'action_row2_1', 'nstate_brake_0',
       'nstate_brake_1', 'nstate_brake_2', 'nstate_thrust_0',
       'nstate_thrust_1', 'nstate_thrust_2', 'nstate_velocity_0',
       'nstate_velocity_1', 'nstate_velocity_2', 'nstate_velocity_3',
       'reward__0', 'state_brake_0', 'state_brake_1', 'state_brake_2',
       'state_thrust_0', 'state_thrust_1', 'state_thrust_2',
       'state_velocity_0', 'state_velocity_1', 'state_velocity_2',
       'state_velocity_3'],
      dtype='object')

,vehicle,driver,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,nstate_brake_0,nstate_brake_1,...,state_brake_0,state_brake_1,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3
episodestart_timestamp,,,,,,,,,,,,,,,,,,,,,
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-07 15:22:43.124431_2023-06-09 16:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-07 15:22:43.124431_2023-06-09 17:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-07 15:22:43.124431_2023-06-09 18:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0
2023-06-07 15:22:43.124431_2023-06-09 19:17:41.649423,MP73,li-changlong,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,...,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0


In [80]:
try:
    ddf_episodes_new = dd.from_pandas(dfs_episodes_new_flat, npartitions=1)
except Exception as e:
    print(e)

# dfs_episodes_new_flat


In [87]:
os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[6, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_episodes_new.to_parquet(
            'eos_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle', 'driver'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

'/home/n/devel/anting_remote/eos/data'

[########################################] | 100% Completed | 101.31 ms


In [88]:
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('eos_episodes/_common_metadata')
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in df.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA


multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
multi_col

multi_idx = [tuple(idx.split('_')) for idx in df.index]
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
multi_idx
df.columns = multi_col
df.index = multi_idx

df = df.set_index(['vehicle', 'driver', df.index])
df
# len(multi_idx)

length: [6, 600, 5]
timezeone: sh
units: ['kph', 'pct', 'pct']
data_propensity: [50, 1, 4]
Dataset Description: MP vehicle from TBox


MultiIndex([( 'action',     'row0',  0),
            ( 'action',     'row0',  1),
            ( 'action',     'row1',  0),
            ( 'action',     'row1',  1),
            ( 'action',     'row2',  0),
            ( 'action',     'row2',  1),
            ( 'nstate',    'brake',  0),
            ( 'nstate',    'brake',  1),
            ( 'nstate',    'brake',  2),
            ( 'nstate',   'thrust',  0),
            ( 'nstate',   'thrust',  1),
            ( 'nstate',   'thrust',  2),
            ( 'nstate', 'velocity',  0),
            ( 'nstate', 'velocity',  1),
            ( 'nstate', 'velocity',  2),
            ( 'nstate', 'velocity',  3),
            ( 'reward',         '',  0),
            (  'state',    'brake',  0),
            (  'state',    'brake',  1),
            (  'state',    'brake',  2),
            (  'state',   'thrust',  0),
            (  'state',   'thrust',  1),
            (  'state',   'thrust',  2),
            (  'state', 'velocity',  0),
            (  '

MultiIndex([('2023-06-02 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-02 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 19:17:41.649423'),
            ('2023-06-07 15:22:43.124431', '2023-06-09 20:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 16:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 17:17:41.649423'),
            ('2023-06-03 15:22:43.124431', '2023-06-09 18:17:41.649423'),
            ('2023-06-03 15:22:43.1244

tuple                                                                      action   
measurement                                                                  row0   
step                                                                            0   
vehicle driver       episodestart               timestamp                           
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  \
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
MP74    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   

tuple                                                                              
measurement                                                                        
step                                                                           1   
vehicle driver       episodestart               timestamp                          
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  12.0  \
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
                                                2023-06-09 20:17:41.649423  12.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423  12.0   
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
                 

### add rows to a partition

In [89]:

ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.index
ddf.npartitions
ddf.divisions
ddf.compute()

Dask Index Structure:
npartitions=4
    object
       ...
       ...
       ...
       ...
Name: episodestart_timestamp, dtype: object
Dask Name: read-parquet-index, 2 graph layers

4

(None, None, None, None, None)

,action_row0_0,action_row0_1,action_row1_0,action_row1_1,action_row2_0,action_row2_1,nstate_brake_0,nstate_brake_1,nstate_brake_2,nstate_thrust_0,...,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle,driver
episodestart_timestamp,,,,,,,,,,,,,,,,,,,,,
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 16:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 17:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 18:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong
2023-06-07 15:22:43.124431_2023-06-09 19:17:41.649423,11.0,12.0,13.0,14.0,15.0,16.0,24.0,25.0,26.0,21.0,...,9.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,MP73,li-changlong


In [90]:
dfs_episode5 = dfs_episode.copy()
dfs_episode5.index = dfs_episode4.index.set_levels(
    [episodestart - pd.Timedelta(9, 'd')], level='episodestart'
)
dfs_episode5.index = dfs_episode4.index.set_levels(
    [[trucks[1].vid], [drivers[0].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode5 = dfs_episode5.swaplevel(0,1, axis=0).swaplevel(1,2,axis=0)
dfs_episode5 = dfs_episode5.reset_index(level=['vehicle', 'driver'])

dfs_episode5['vehicle'] = dfs_episode5['vehicle'].astype('category')
dfs_episode5['driver'] = dfs_episode5['driver'].astype('category')
dfs_episode5

tuple                                                 vehicle      driver   
measurement                                                                 
step                                                                        
episodestart               timestamp                                        
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423    MP74  wang-cheng  \
                           2023-06-09 17:17:41.649423    MP74  wang-cheng   
                           2023-06-09 18:17:41.649423    MP74  wang-cheng   
                           2023-06-09 19:17:41.649423    MP74  wang-cheng   
                           2023-06-09 20:17:41.649423    MP74  wang-cheng   

tuple                                                 state                    
measurement                                           brake           thrust   
step                                                      0    1    2      0   
episodestart               timestamp                                           
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   7.0  8.0  9.0    4.0  \
                           2023-06-09 17:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 18:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 19:17:41.649423   7.0  8.0  9.0    4.0   
                           2023-06-09 20:17:41.649423   7.0  8.0  9.0    4.0   

tuple                                                                           
measurement                                                     velocity        
step                                                     1    2        0    1   
episodestart               timestamp                                            
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  5.0  6.0      0.0  1.0  \
                           2023-06-09 17:17:41.649423  5.0  6.0      0.0  1.0   
                           2023-06-09 18:17:41.649423  5.0  6.0      0.0  1.0   
                           2023-06-09 19:17:41.649423  5.0  6.0      0.0  1.0   
                           2023-06-09 20:17:41.649423  5.0  6.0      0.0  1.0   

tuple                                                  ... nstate               
measurement                                            ...  brake               
step                                                   ...      0     1     2   
episodestart               timestamp                   ...                      
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  ...   24.0  25.0  26.0  \
                           2023-06-09 17:17:41.649423  ...   24.0  25.0  26.0   
                           2023-06-09 18:17:41.649423  ...   24.0  25.0  26.0   
                           2023-06-09 19:17:41.649423  ...   24.0  25.0  26.0   
                           2023-06-09 20:17:41.649423  ...   24.0  25.0  26.0   

tuple                                                                      
measurement                                           thrust               
step                                                       0     1     2   
episodestart               timestamp                                       
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   21.0  22.0  23.0  \
                           2023-06-09 17:17:41.649423   21.0  22.0  23.0   
                           2023-06-09 18:17:41.649423   21.0  22.0  23.0   
                           2023-06-09 19:17:41.649423   21.0  22.0  23.0   
                           2023-06-09 20:17:41.649423   21.0  22.0  23.0   

tuple                                                                        
measurement                                           velocity               
step                                                         0     1     2   
episodestart               timestamp                                         
2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423     17.0  18.0  19.0  \
                           2023-06-09 17:17:41.649

#### decoding single dataframe of multiindex into dask single index dataframe

In [91]:
df_episode_new = dfs_episode5.copy()
df_episode_new.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}'
    for x in dfs_episode5.columns.to_flat_index()
]
df_episode_new.index = [
    f'{x[0]}_{x[1]}' if x[1] != '' else f'x{0}'
    for x in dfs_episode5.index.to_flat_index()
]
# df_episode_new.dtypes
df_episode_new

,vehicle,driver,state_brake_0,state_brake_1,state_brake_2,state_thrust_0,state_thrust_1,state_thrust_2,state_velocity_0,state_velocity_1,...,nstate_brake_0,nstate_brake_1,nstate_brake_2,nstate_thrust_0,nstate_thrust_1,nstate_thrust_2,nstate_velocity_0,nstate_velocity_1,nstate_velocity_2,nstate_velocity_3
2023-06-02 15:22:43.124431_2023-06-09 16:17:41.649423,MP74,wang-cheng,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,...,24.0,25.0,26.0,21.0,22.0,23.0,17.0,18.0,19.0,20.0
2023-06-02 15:22:43.124431_2023-06-09 17:17:41.649423,MP74,wang-cheng,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,...,24.0,25.0,26.0,21.0,22.0,23.0,17.0,18.0,19.0,20.0
2023-06-02 15:22:43.124431_2023-06-09 18:17:41.649423,MP74,wang-cheng,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,...,24.0,25.0,26.0,21.0,22.0,23.0,17.0,18.0,19.0,20.0
2023-06-02 15:22:43.124431_2023-06-09 19:17:41.649423,MP74,wang-cheng,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,...,24.0,25.0,26.0,21.0,22.0,23.0,17.0,18.0,19.0,20.0
2023-06-02 15:22:43.124431_2023-06-09 20:17:41.649423,MP74,wang-cheng,7.0,8.0,9.0,4.0,5.0,6.0,0.0,1.0,...,24.0,25.0,26.0,21.0,22.0,23.0,17.0,18.0,19.0,20.0


In [93]:

ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
df = ddf.compute()
df.dtypes

action_row0_0         float64
action_row0_1         float64
action_row1_0         float64
action_row1_1         float64
action_row2_0         float64
action_row2_1         float64
nstate_brake_0        float64
nstate_brake_1        float64
nstate_brake_2        float64
nstate_thrust_0       float64
nstate_thrust_1       float64
nstate_thrust_2       float64
nstate_velocity_0     float64
nstate_velocity_1     float64
nstate_velocity_2     float64
nstate_velocity_3     float64
reward__0             float64
state_brake_0         float64
state_brake_1         float64
state_brake_2         float64
state_thrust_0        float64
state_thrust_1        float64
state_thrust_2        float64
state_velocity_0      float64
state_velocity_1      float64
state_velocity_2      float64
state_velocity_3      float64
vehicle              category
driver               category
dtype: object

In [94]:
df_list = []
for n in range(ddf.npartitions):
    df_list.append(ddf.partitions[n])
len(df_list)

4

In [95]:
for i, ddf in enumerate(df_list):
    # if dfs_episode_new['vehicle']
    print(f"{i}")
    # df = ddf.compute()
    # ddf.columns
    # ddf.dtypes
    vehicle = ddf['vehicle'].compute()[0]
    driver = ddf['driver'].compute()[0]
    # df_episode_new = df_episode_new.append(ddf)
    if (
        df_episode_new['vehicle'][0] == vehicle
        and df_episode_new['driver'][0] == driver
    ):
        print(
            f'hit {i}{"st" if i==1 else "nd" if i==2 else "rd" if i==3 else "th"} partition'
        )
        df_list[i] = dd.concat([ddf, df_episode_new], axis=0)
        # append ddf to dfs_episode_new

    #     c_ep = df_episode_new['ep
    #     dfs_episode_new = dfs_episode_newappend(ddf)
    # print(f"splicing dfs_episode_new {c_ep} with


0
1
2
3
hit 3rd partition


In [96]:

ddf_all = dd.concat(df_list, axis=0, join='outer')
# ddf_all.compute()os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[8, 800, 8],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_all.to_parquet(
            'eos_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle', 'driver'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)


[########################################] | 100% Completed | 101.94 ms


In [99]:
# ddf_all.compute()
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('ddf_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in ddf.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
df.columns = multi_col

multi_idx = [tuple(idx.split('_')) for idx in ddf.index]
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
df.index = multi_idx
df = df.set_index(['vehicle', 'driver', df.index])
df
df.columns
df.index
# len(multi_idx)


length: [6, 600, 5]
timezeone: sh
units: ['kph', 'pct', 'pct']
data_propensity: [50, 1, 4]
Dataset Description: MP vehicle from TBox


tuple                                                                      action   
measurement                                                                  row0   
step                                                                            0   
vehicle driver       episodestart               timestamp                           
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  \
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
MP74    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   

tuple                                                                              
measurement                                                                        
step                                                                           1   
vehicle driver       episodestart               timestamp                          
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  12.0  \
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
            

MultiIndex([('action',     'row0', 0),
            ('action',     'row0', 1),
            ('action',     'row1', 0),
            ('action',     'row1', 1),
            ('action',     'row2', 0),
            ('action',     'row2', 1),
            ('nstate',    'brake', 0),
            ('nstate',    'brake', 1),
            ('nstate',    'brake', 2),
            ('nstate',   'thrust', 0),
            ('nstate',   'thrust', 1),
            ('nstate',   'thrust', 2),
            ('nstate', 'velocity', 0),
            ('nstate', 'velocity', 1),
            ('nstate', 'velocity', 2),
            ('nstate', 'velocity', 3),
            ('reward',         '', 0),
            ( 'state',    'brake', 0),
            ( 'state',    'brake', 1),
            ( 'state',    'brake', 2),
            ( 'state',   'thrust', 0),
            ( 'state',   'thrust', 1),
            ( 'state',   'thrust', 2),
            ( 'state', 'velocity', 0),
            ( 'state', 'velocity', 1),
            ( 'state', 'v

MultiIndex([('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-02 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73', 'li-changlong', '2023-06-07 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15:22:43.124431', ...),
            ('MP73',   'wang-cheng', '2023-06-03 15

In [101]:
df = df.sort_index()
df

tuple                                                                      action   
measurement                                                                  row0   
step                                                                            0   
vehicle driver       episodestart               timestamp                           
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0  \
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-07 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-03 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-09 15:17:41.649423 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
MP74    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
        wang-cheng   2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   
                     2023-06-04 15:22:43.124431 2023-06-09 16:17:41.649423   11.0   
                                                2023-06-09 17:17:41.649423   11.0   
                                                2023-06-09 18:17:41.649423   11.0   
                                                2023-06-09 19:17:41.649423   11.0   
                                                2023-06-09 20:17:41.649423   11.0   

tuple                                                                              
measurement                                                                        
step                                                                           1   
vehicle driver       episodestart               timestamp                          
MP73    li-changlong 2023-06-02 15:22:43.124431 2023-06-09 16:17:41.649423  12.0  \
                                                2023-06-09 17:17:41.649423  12.0   
                                                2023-06-09 18:17:41.649423  12.0   
                                                2023-06-09 19:17:41.649423  12.0   
            